# DEPENDÊNCIAS

In [ ]:
! pip install spacy
! pip install nltk
! pip install pandas
! pip install pdfplumber
! pip install chromadb
! pip install sentence-transformers
! pip install openpyxl
! pip install python-docx
! python -m spacy download pt_core_news_sm
! pip install langchain-text-splitters

# Carrega PDF

In [9]:
import pdfplumber

pdf_path = "./documents/sample_portugues.pdf"

def load_pdf(filepath):
    with pdfplumber.open(filepath) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text() or ""
    return text

raw_text = load_pdf(pdf_path)

print("Prévia do texto lido:\n")
print(raw_text[:800])


Prévia do texto lido:

Título: Introdução ao Machine Learning
Machine learning é um campo da inteligência artificial que desenvolve algoritmos
capazes de aprender padrões a partir de dados. Os principais tipos incluem
aprendizado supervisionado, aprendizado não supervisionado e aprendizado por
reforço.
Aprendizado supervisionado envolve treinar um modelo com dados rotulados,
como prever preços de casas com base em características como tamanho,
localização e número de quartos. Aprendizado não supervisionado detecta
padrões ocultos em dados não rotulados, como segmentação de clientes em
marketing. Aprendizado por reforço ensina agentes a tomar decisões em
ambientes dinâmicos para maximizar recompensas.
Redes neurais profundas são usadas em visão computacional e processamento
de linguagem natural. Modelos como CNNs


# PLN

## Lowercasing

In [10]:
text_lower = raw_text.lower()

print("Texto em lowercase (prévia):\n")
print(text_lower[:800])


Texto em lowercase (prévia):

título: introdução ao machine learning
machine learning é um campo da inteligência artificial que desenvolve algoritmos
capazes de aprender padrões a partir de dados. os principais tipos incluem
aprendizado supervisionado, aprendizado não supervisionado e aprendizado por
reforço.
aprendizado supervisionado envolve treinar um modelo com dados rotulados,
como prever preços de casas com base em características como tamanho,
localização e número de quartos. aprendizado não supervisionado detecta
padrões ocultos em dados não rotulados, como segmentação de clientes em
marketing. aprendizado por reforço ensina agentes a tomar decisões em
ambientes dinâmicos para maximizar recompensas.
redes neurais profundas são usadas em visão computacional e processamento
de linguagem natural. modelos como cnns


## Remover caracteres especiais

In [ ]:
import re
import unicodedata

def remove_special_chars(text):
    # Normaliza acentos (NFKD)
    text = unicodedata.normalize("NFKD", text)
    text = text.encode("ASCII", "ignore").decode("utf-8")

    # Remove caracteres que não sejam letras ou espaço
    text = re.sub(r"[^a-z0-9.,\s]", " ", text)

    # Remove múltiplos espaços
    text = re.sub(r"\s+", " ", text)

    return text.strip()

text_clean = remove_special_chars(text_lower)

print("Texto sem caracteres especiais (prévia):\n")
print(text_clean[:800])


Texto sem caracteres especiais (prévia):

titulo introducao ao machine learning machine learning e um campo da inteligencia artificial que desenvolve algoritmos capazes de aprender padroes a partir de dados. os principais tipos incluem aprendizado supervisionado, aprendizado nao supervisionado e aprendizado por reforco. aprendizado supervisionado envolve treinar um modelo com dados rotulados, como prever precos de casas com base em caracteristicas como tamanho, localizacao e numero de quartos. aprendizado nao supervisionado detecta padroes ocultos em dados nao rotulados, como segmentacao de clientes em marketing. aprendizado por reforco ensina agentes a tomar decisoes em ambientes dinamicos para maximizar recompensas. redes neurais profundas sao usadas em visao computacional e processamento de linguagem natural. modelos como cnns 


## Remover Stopwords

In [72]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Baixar recursos necessários (roda apenas 1 vez)
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("stopwords")

stopwords_pt = set(stopwords.words("portuguese"))

tokens = word_tokenize(text_clean, language="portuguese")

tokens_no_stop = [t for t in tokens if t not in stopwords_pt]

print("Tokens sem stopwords (primeiros 30):\n")
print(tokens_no_stop[:30])


Tokens sem stopwords (primeiros 30):

['titulo', 'introducao', 'machine', 'learning', 'machine', 'learning', 'campo', 'inteligencia', 'artificial', 'desenvolve', 'algoritmos', 'capazes', 'aprender', 'padroes', 'partir', 'dados', '.', 'principais', 'tipos', 'incluem', 'aprendizado', 'supervisionado', ',', 'aprendizado', 'nao', 'supervisionado', 'aprendizado', 'reforco', '.', 'aprendizado']


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\GamingPc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\GamingPc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\GamingPc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Lemmatização

In [79]:
import spacy

# Carregar modelo de português
nlp = spacy.load("pt_core_news_sm")

def lemmatize_text(tokens):
    doc = nlp(" ".join(tokens))
    return [token.lemma_ for token in doc]

lemmas = " ".join(lemmatize_text(tokens_no_stop))

print("Lemmas (primeiros 30):\n")
print(lemmas[:300])


Lemmas (primeiros 30):

titulo introducao machine Learning machine learning Campo inteligencia artificial desenvolver algoritmos capaz aprender padroe partir dado . principal tipo incluir aprendizar supervisionar , aprendizar nao supervisionar aprendizar reforco . aprendizar supervisionar envolver treinar modelo dado rotul


## Stemmatização

In [80]:
# --- STEMMATIZAÇÃO (alternativa à lematização) ---
from nltk.stem.snowball import SnowballStemmer

# Criar stemmer para português
stemmer = SnowballStemmer("portuguese")

# Aplicar stemmatização nos tokens sem stopwords
stems = " ".join([stemmer.stem(token) for token in tokens_no_stop])

print("Stems (primeiros 30):\n")
print(stems[:300])


Stems (primeiros 30):

titul introduca machin learning machin learning camp inteligenc artificial desenvolv algoritm capaz aprend padro part dad . princip tip inclu aprendiz supervision , aprendiz nao supervision aprendiz reforc . aprendiz supervision envolv trein model dad rotul , prev prec cas bas caracterist tamanh , l


# Chunking


###  MANUAL

#### Por tamanho fixo

In [47]:
# --- CHUNKING POR TAMANHO FIXO (CARACTERES) ---
def chunk_by_chars(text, chunk_size=500, overlap=50):
    chunks = []
    start = 0
    
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(chunk)
        start += chunk_size - overlap
    
    return chunks

chunks_chars = chunk_by_chars(lemmas, chunk_size=100, overlap=10)

print(f"Total de chunks: {len(chunks_chars)}")
print("Exemplo de chunk:\n")
print(chunks_chars[0][:300])


Total de chunks: 9
Exemplo de chunk:

titulo introducao machine Learning machine learning Campo inteligencia artificial desenvolver algori


#### Por tokens

In [49]:
# --- CHUNKING POR TOKENS ---
from nltk.tokenize import word_tokenize

def chunk_by_tokens(text, token_limit=200, overlap=30):
    tokens = word_tokenize(text)
    chunks = []
    start = 0

    while start < len(tokens):
        end = start + token_limit
        chunk_tokens = tokens[start:end]
        chunks.append(" ".join(chunk_tokens))
        start += token_limit - overlap
    
    return chunks

chunks_tokens = chunk_by_tokens(lemmas, token_limit=200, overlap=30)

print(f"Total de chunks: {len(chunks_tokens)}")
print("Exemplo de chunk:\n")
print(chunks_tokens[0][:300])


Total de chunks: 1
Exemplo de chunk:

titulo introducao machine Learning machine learning Campo inteligencia artificial desenvolver algoritmos capaz aprender padroe partir dado principal tipo incluir aprendizar supervisionar aprendizar nao supervisionar aprendizar reforco aprendizar supervisionar envolver treinar modelo dado rotular pre


#### Por parágrafos

In [53]:
# --- CHUNKING POR PARÁGRAFOS ---
def chunk_by_paragraph(text):
    paragraphs = [p.strip() for p in text.split("\n") if p.strip()]
    return paragraphs

chunks_paragraphs = chunk_by_paragraph(lemmas)

print(f"Total de chunks: {len(chunks_paragraphs)}")
print("Primeiro parágrafo:\n")
print(chunks_paragraphs[0][:300])


Total de chunks: 1
Primeiro parágrafo:

titulo introducao machine Learning machine learning Campo inteligencia artificial desenvolver algoritmos capaz aprender padroe partir dado principal tipo incluir aprendizar supervisionar aprendizar nao supervisionar aprendizar reforco aprendizar supervisionar envolver treinar modelo dado rotular pre


#### Por Sentenças

In [54]:
# --- CHUNKING POR SENTENÇAS ---
from nltk.tokenize import sent_tokenize

def chunk_by_sentences(text, max_sentences=5):
    sentences = sent_tokenize(text)
    chunks = []

    for i in range(0, len(sentences), max_sentences):
        chunk = " ".join(sentences[i:i + max_sentences])
        chunks.append(chunk)

    return chunks

chunks_sentences = chunk_by_sentences(lemmas, max_sentences=5)

print(f"Total de chunks: {len(chunks_sentences)}")
print("Exemplo de chunk:\n")
print(chunks_sentences[0][:300])


Total de chunks: 1
Exemplo de chunk:

titulo introducao machine Learning machine learning Campo inteligencia artificial desenvolver algoritmos capaz aprender padroe partir dado principal tipo incluir aprendizar supervisionar aprendizar nao supervisionar aprendizar reforco aprendizar supervisionar envolver treinar modelo dado rotular pre


#### Híbrido

In [57]:
# --- CHUNKING HÍBRIDO (SENTENÇAS ATÉ LIMITE DE CARACTERES) ---
from nltk.tokenize import sent_tokenize

def hybrid_sentence_chunk(text, max_chars=500):
    sentences = sent_tokenize(text)
    chunks = []
    current = ""

    for sent in sentences:
        if len(current) + len(sent) <= max_chars:
            current += " " + sent
        else:
            chunks.append(current.strip())
            current = sent
    
    if current:
        chunks.append(current.strip())
    
    return chunks

chunks_hybrid = hybrid_sentence_chunk(lemmas, max_chars=500)

print(f"Total de chunks: {len(chunks_hybrid)}")
print("Exemplo de chunk:\n")
print(chunks_hybrid[1][:300])


Total de chunks: 2
Exemplo de chunk:

titulo introducao machine Learning machine learning Campo inteligencia artificial desenvolver algoritmos capaz aprender padroe partir dado principal tipo incluir aprendizar supervisionar aprendizar nao supervisionar aprendizar reforco aprendizar supervisionar envolver treinar modelo dado rotular pre


#### Recursívo

In [58]:
# --- CHUNKING RECURSIVO ---

def recursive_chunk(text, max_size=500, overlap=50):
    """
    Implementação simplificada do Recursive Character Text Splitter
    estilo LangChain.
    """

    # ordem de separadores: do mais forte ao mais fraco
    separators = ["\n\n", "\n", ". ", " "]

    def split_text(text, separators):
        """Divide texto usando o primeiro separador disponível."""
        for sep in separators:
            if sep in text:
                return text.split(sep), sep
        return [text], None  # nenhum separador encontrado

    # ---------- função recursiva -----------

    def _recursive(text):
        if len(text) <= max_size:
            return [text]

        parts, sep = split_text(text, separators)

        # Se o texto não tiver nenhum dos separadores
        if sep is None:
            # chunk por caracteres mesmo
            return [
                text[i:i+max_size]
                for i in range(0, len(text), max_size)
            ]

        chunks = []
        current = ""

        for part in parts:
            # tenta adicionar a parte atual
            if len(current) + len(part) + len(sep) <= max_size:
                current += part + sep
            else:
                # se a parte é maior que o limite,
                # precisamos dividir recursivamente
                if len(part) > max_size:
                    chunks.extend(_recursive(part))
                else:
                    chunks.append(current.strip())
                    current = part + sep

        if current.strip():
            chunks.append(current.strip())

        # aplicar overlap
        if overlap > 0:
            chunks_with_overlap = []
            for i in range(len(chunks)):
                chunk = chunks[i]
                if i > 0:
                    chunk = chunks[i-1][-overlap:] + " " + chunk
                chunks_with_overlap.append(chunk)
            chunks = chunks_with_overlap

        return chunks

    # ---------------------------------------

    return _recursive(text)


# ---- executando no seu texto limpo ----
chunks_recursive = recursive_chunk(lemmas, max_size=500, overlap=50)

print(f"Total de chunks gerados: {len(chunks_recursive)}")
print("\nExemplo de chunk:\n")
print(chunks_recursive[0][:400])


Total de chunks gerados: 2

Exemplo de chunk:

titulo introducao machine Learning machine learning Campo inteligencia artificial desenvolver algoritmos capaz aprender padroe partir dado principal tipo incluir aprendizar supervisionar aprendizar nao supervisionar aprendizar reforco aprendizar supervisionar envolver treinar modelo dado rotular prever preco casa base caracteristico tamanho localizacao numero quarto aprendizar nao supervisionar de


### COM LIBRARY

#### Por Tamanho Fixo

In [119]:
# --- CHARACTER TEXT SPLITTER ---
from langchain_text_splitters import CharacterTextSplitter

char_splitter = CharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=10,
    separator=".",
    length_function=len
)

chunks_char = char_splitter.split_text(lemmas)

print(f"Total de chunks: {len(chunks_char)}")
print(chunks_char)


Total de chunks: 7
['titulo introducao machine Learning machine learning Campo inteligencia artificial desenvolver algoritmos capaz aprender padroe partir dado', 'principal tipo incluir aprendizar supervisionar , aprendizar nao supervisionar aprendizar reforco', 'aprendizar supervisionar envolver treinar modelo dado rotular , prever preco casa base caracteristico tamanho , localizacaar numero quarto', 'aprendizar nao supervisionar detecta padroe oculto dar nao rotular , segmentacao cliente Marketing', 'aprendizar reforco ensina agente tomar decisoes ambiente dinamico maximizar recompensa', 'rede neural profundo sao usar visao computacional processamento linguagem natural', 'modelo cnns transformer revolucionar tarefa traducao automatica , reconhecimento imagem geracaar texto']


#### Recursívo

In [120]:
# --- RECURSIVE CHARACTER TEXT SPLITTER ---
from langchain_text_splitters import RecursiveCharacterTextSplitter

recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=10,
    separators=["\n\n", "\n", ".", ",", " ", ""]
)

chunks_recursive = recursive_splitter.split_text(lemmas)

print(f"Total de chunks: {len(chunks_recursive)}")
print(chunks_recursive)


Total de chunks: 7
['titulo introducao machine Learning machine learning Campo inteligencia artificial desenvolver algoritmos capaz aprender padroe partir dado', '. principal tipo incluir aprendizar supervisionar , aprendizar nao supervisionar aprendizar reforco', '. aprendizar supervisionar envolver treinar modelo dado rotular , prever preco casa base caracteristico tamanho , localizacaar numero quarto', '. aprendizar nao supervisionar detecta padroe oculto dar nao rotular , segmentacao cliente Marketing', '. aprendizar reforco ensina agente tomar decisoes ambiente dinamico maximizar recompensa', '. rede neural profundo sao usar visao computacional processamento linguagem natural', '. modelo cnns transformer revolucionar tarefa traducao automatica , reconhecimento imagem geracaar texto .']
